In [1]:
import os
import pickle
import warnings
from datetime import datetime

import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import xgboost
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

warnings.filterwarnings("ignore")

/Users/mdurango/Proyect/Mlops-platzi/.venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/mdurango/Proyect/Mlops-platzi/.venv/lib/python3.9/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


Ahora lo que vamos a hacer es ir a la UI y simulemos que algún cientifico de datos o miembro del equipo decide registrar ciertos podemos en "staging", y más adelante nosotros como ingenieros de ML tomamos la decisión de cual modelo registrar para producción bajo las métricas de performance obtenidas y las pruebas. Es muy importante, hacer las pruebas de validación antes de considerar un modelo en producción. Entre ellas se encuentran:

#### 1. Pruebas de Funcionalidad:

* Funcionalidad del Modelo: Asegurarse de que el modelo funcione correctamente en el entorno de producción con los datos reales (Se puede simular con cierta parte de la data o toda idealmente).
* Integración del Modelo: Verificar que el modelo integrado en la infraestructura de producción responda y se comunique adecuadamente con otros componentes del sistema (partes del Pipeline).

####  2. Pruebas de Rendimiento:
* Rendimiento del Modelo: Evaluar el rendimiento del modelo en términos de velocidad de inferencia, uso de recursos (CPU, memoria), y latencia para garantizar que cumple con los requisitos de producción. (El tiempo es un factor muy importante en la generación de soluciones de negocio)
* Escala y Carga: Probar el modelo bajo cargas esperadas para asegurar que el sistema es escalable y puede manejar la demanda prevista. (Orar para que el Kernel no muera con la carga, de lo contrario mirar formas de optimizar el código, hacer carga de datos por chuncks, paralelizar procesos, etc)

#### 3. Pruebas de Robustez:
* Robustez de estrés: Evaluar el comportamiento del modelo en condiciones adversas, como datos de entrada no esperados, valores faltantes, valores atípicos, etc. (Esto es muy importante, ya que en la vida real los datos no son perfectos, y es muy probable que el modelo se caiga si no se tiene en cuenta este factor por lo que generar pruebas unitarias para este tipo de casos es muy importante). 


### Making predictions from mlflow artifacts

Predict on Pandas DataFrame: Para ello debemos tener la misma representación de datos que hemos usado para entrenar el modelo, es decir, las mismas columnas y el mismo orden. Por lo que vamos a usar un par de funciones que hemos venido implementando. 

In [2]:
def load_pickle(filename):
    """ 
    This function loads a pickle file from the data_processed folder
    and returns the data
    Args:
        filename (str): name of the file to load
    Returns:
        data (pd.DataFrame): data loaded from the pickle file
    """
    filepath = os.path.join("data", "data_processed", f"{filename}.pkl")
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
    return data

In [3]:
X_train, y_train = load_pickle("train")
X_test, y_test = load_pickle("test")

In [5]:
y_train.shape

(13269,)

In [6]:
X_train.toarray().shape

(13269, 13692)

In [7]:
assert X_train.toarray().shape[0] == y_train.shape[0]

In [8]:


# Establecer la URI de tracking
mlflow.set_tracking_uri('sqlite:///mlflow.db')

# Ahora puedes cargar tu modelo
#logged_model = './mlruns/1/e1aedba8c8ea485c91f81b8b124c269f/artifacts/models/LogisticRegression'
logged_model = "logged_model = 'runs:/5f13a7a3b4804585afed5d5a10a82f7b/models/LogisticRegression'"
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [9]:
# get info from the loaded model
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models/LogisticRegression
  flavor: mlflow.sklearn
  run_id: e1aedba8c8ea485c91f81b8b124c269f

In [10]:
lr_model = mlflow.sklearn.load_model(logged_model)

In [11]:
lr_model.predict(X_test)

array(['2', '2', '2', ..., '2', '2', '2'], dtype=object)

## Model registry and state transitions

<p align="center">
  <img src="https://pbs.twimg.com/media/EoOBoWyWEAAA8In.jpg" width="400" style="display: block; margin: auto;">
</p>


* Tener en cuenta el tiempo de ejecución

* Métricas obtenidas

* Size del tamaño 

* Las pruebas que anteriormente mencionamos. 


In [12]:
from mlflow.tracking import MlflowClient

In [18]:
mlflow.set_tracking_uri("sqlite:///backend.db")
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [19]:
runs = mlflow.search_runs()
# Extrae los IDs únicos de los experimentos
experiment_ids = runs['experiment_id'].unique()

In [20]:
experiment_ids

array([], dtype=float64)

In [16]:
client.create_experiment("regression_models")

'1'

In [17]:
runs = client.search_runs(experiment_ids=["1"])
for run in runs:
    print(run.info)


In [ ]:
runs= client.search_runs(
    experiment_ids='1',
    filter_string="", #se puede usar un tag en especial o algún parámetro de interés ej: 'tags.model = "lasso_regression" '
    run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse_valid ASC"]

)

In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse_valid: {run.data.metrics['rmse_valid']}")

### Promote a model

In [ ]:
run_id = "a90d6cc190014a21ba5a0dc4de13f94e"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri, "regressor_model")

In [ ]:
model_name = "regressor_model"
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f" version: {version.version} , actual stage: {version._current_stage}")

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Staging"
)

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production"
)

Cuando vayas a cambiar el staging de un modelo a otro estado, podemos también añadir descripciones: 

In [ ]:
client.update_model_version(
    name=model_name, 
    version=2,
    description=f"The model version {2} was transitioned to Production on {datetime.today().date()}"
)

In [ ]:
type(test.get_label())

In [ ]:
test.get_label()

In [ ]:
test.get_data()

### Testing a register model: 

De esta forma se deben hacer las pruebas con el pipeline de producción para asegurar que el modelo se comporta como se espera, además de validar la infraestructura y las pruebas que recién habíamos mencionado. 

In [ ]:
# OJO: Es importante que recuerdes los formatos permitidos por mlflow load_model, como dataframes, numpy arrays, etc.
def testint_model_from_mlflow(model_name: str, stage:str, X_test: xgboost.core.DMatrix, Y_test: np.ndarray):
    """this function tests a model from mlflow
    Args:
        model_name (str): name of the model
        stage (str): stage of the model
        X_test (scipy.sparse._csr.csr_matrix): test data
        Y_test (scipy.sparse._csr.csr_matrix): test target
    Returns:
        float: rmse of the model
    
    """
    model_uri = f"models:/{model_name}/{stage}"
    model = mlflow.pyfunc.load_model(model_uri)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(Y_test, y_pred, squared=False)
    return {"rmse": rmse}

In [ ]:
%time
testint_model_from_mlflow(model_name= "regressor_model", stage="Production", X_test=test, Y_test=test.get_label())


#### Ventajas de MLflow:

* Gestión de Ciclo de Vida: Facilita el seguimiento de experimentos, versionado de modelos y reproducción de resultados.
* Interoperabilidad: Es compatible con múltiples frameworks de aprendizaje automático y se integra fácilmente en flujos de trabajo existentes.
* Abierto y Modular: Ofrece una arquitectura modular que permite la flexibilidad y personalización.
* Trazabilidad y Reproducibilidad: Registra métricas, parámetros y artefactos para reproducir modelos y resultados.
* Comunidad Activa: Amplia comunidad de usuarios y contribuciones continuas.


#### Desventajas de MLflow:
* Complejidad para Grandes Volúmenes de Datos: Puede enfrentar dificultades al manejar grandes volúmenes de datos o flujos de trabajo muy complejos.
* Curva de Aprendizaje: Requiere tiempo para familiarizarse con todas sus funcionalidades y componentes.
Limitaciones en Algunas Funcionalidades: Algunas funcionalidades pueden no ser tan avanzadas o flexibles como en otras herramientas especializadas.


### Alternativas a MLflow:
* TensorBoard: Enfoque específico para TensorFlow, útil para visualizar gráficamente métricas, grafos de modelos y más.
* DVC (Data Version Control): Se enfoca en versionado de datos y modelos, y gestión de experimentos.
* Comet.ml: Ofrece seguimiento de experimentos, colaboración y visualización de manera similar a MLflow.
* Weights & Biases: Ofrece seguimiento de experimentos, colaboración y visualización de manera similar a MLflow.


#### Conclusiones

* Podemos trackear metadata

* Registrar modelos

* Obtener los requirimientos del ambiente de desarrollo donde fue entrenado los modelos

* Podemos hacer un seguimiento de los modelos y compararlos de forma fácil y amigable con la interfaz de MLflow y en código

* Hacer transiciones de estados de los modelos

* Añadir anotaciones o descripciones 

So, ya sabes incluír el registro de modelos en nuestro pipeline de ML, ahora vamos a aprender de workflows y tasks. ¡Te espero en la próxima clase! :) 